# Part 1 项目介绍

## 1.1 项目背景
CIFAR-10是一个经典的计算机视觉数据集,包含10个类别的60000张32x32彩色图像,其中训练集50000张,测试集10000张。这10个类别分别是:飞机(airplane)、汽车(automobile)、鸟(bird)、猫(cat)、鹿(deer)、狗(dog)、青蛙(frog)、马(horse)、船(ship)和卡车(truck)。

## 1.2 项目目标

本项目旨在对比传统机器学习方法和深度学习方法在图像分类任务上的性能差异:

1. **传统方法**: 使用HOG特征提取 + PCA降维 + SVM分类器
2. **深度学习方法**: 使用卷积神经网络(CNN)

通过对比实验,我们将分析两种方法在准确率、训练时间等方面的差异,深入理解深度学习相比传统方法的优势。

## 1.3 技术路线

- **Part 2**: 特征提取和降维(HOG + PCA + t-SNE可视化)
- **Part 3**: 传统模型实验(SVM分类器)
- **Part 4**: 深度模型实验(CNN)
- **Part 5**: 总结与对比分析

# Part 2 特征提取和降维

本部分将提取图像的HOG特征并进行降维处理,为后续的传统机器学习模型做准备。

## 2.1 特征提取

### 2.1.1 HOG特征介绍

**HOG (Histogram of Oriented Gradients)**,即方向梯度直方图,是一种在计算机视觉和图像处理中用来进行物体检测的特征描述子。

**基本原理**:
1. **梯度计算**: 计算图像中每个像素的梯度方向和幅值
2. **单元格划分**: 将图像划分为小的单元格(cells)
3. **方向直方图**: 在每个单元格内统计梯度方向的直方图
4. **块归一化**: 将相邻的单元格组成块(blocks),并进行归一化

**优点**:
- 对光照变化不敏感
- 能够捕捉边缘和形状信息
- 具有良好的几何不变性

**参数设置**:
- `orientations=9`: 梯度方向被分为9个bin
- `pixels_per_cell=(8, 8)`: 每个单元格为8x8像素
- `cells_per_block=(2, 2)`: 每个块包含2x2个单元格

### 2.1.2 提取HOG特征
数据可视化

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
from skimage.feature import hog
from skimage import color
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子以确保结果可复现
np.random.seed(42)

print("库导入成功!")

In [ ]:
# 加载训练数据标签
data_dir = 'Data/cifar-10'
train_labels_df = pd.read_csv(os.path.join(data_dir, 'trainLabels.csv'))

print(f"训练集样本数: {len(train_labels_df)}")
print(f"\n类别分布:")
print(train_labels_df['label'].value_counts())
print(f"\n前5行数据:")
print(train_labels_df.head())

In [ ]:
# 可视化部分样本
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
fig.suptitle('CIFAR-10 样本展示', fontsize=16)

# 随机选择10个样本
sample_ids = np.random.choice(train_labels_df['id'].values, 10, replace=False)

for idx, (ax, img_id) in enumerate(zip(axes.flatten(), sample_ids)):
    img_path = os.path.join(data_dir, 'train', f'{img_id}.png')
    img = Image.open(img_path)
    label = train_labels_df[train_labels_df['id'] == img_id]['label'].values[0]
    
    ax.imshow(img)
    ax.set_title(f'{label}', fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 定义HOG特征提取函数
def extract_hog_features(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    """
    提取图像的HOG特征
    
    参数:
        image: RGB图像 (H, W, 3)
        orientations: 梯度方向的bin数量
        pixels_per_cell: 每个cell的像素大小
        cells_per_block: 每个block包含的cell数量
    
    返回:
        hog_features: HOG特征向量
    """
    # 将RGB图像转换为灰度图
    if len(image.shape) == 3:
        gray_image = color.rgb2gray(image)
    else:
        gray_image = image
    
    # 提取HOG特征
    hog_features = hog(gray_image, 
                       orientations=orientations,
                       pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block,
                       visualize=False,
                       feature_vector=True)
    
    return hog_features

# 测试HOG特征提取
test_img_path = os.path.join(data_dir, 'train', '1.png')
test_img = np.array(Image.open(test_img_path))
test_hog = extract_hog_features(test_img)

print(f"原始图像形状: {test_img.shape}")
print(f"HOG特征维度: {test_hog.shape}")
print(f"HOG特征前10个值: {test_hog[:10]}")

In [ ]:
# 批量提取HOG特征
# 为了演示,我们使用前10000个样本(可以调整以使用全部数据)
n_samples = 10000  # 使用前10000个样本进行演示,完整实验可设为50000

print(f"开始提取 {n_samples} 个样本的HOG特征...")

hog_features_list = []
labels_list = []

for i in range(n_samples):
    if (i + 1) % 1000 == 0:
        print(f"已处理: {i + 1}/{n_samples}")
    
    # 获取图像ID和标签
    img_id = train_labels_df.iloc[i]['id']
    label = train_labels_df.iloc[i]['label']
    
    # 加载图像
    img_path = os.path.join(data_dir, 'train', f'{img_id}.png')
    img = np.array(Image.open(img_path))
    
    # 提取HOG特征
    hog_feat = extract_hog_features(img)
    
    hog_features_list.append(hog_feat)
    labels_list.append(label)

# 转换为numpy数组
X_hog = np.array(hog_features_list)
y = np.array(labels_list)

print(f"\n特征提取完成!")
print(f"HOG特征矩阵形状: {X_hog.shape}")
print(f"标签数组形状: {y.shape}")
print(f"\n类别分布:")
unique, counts = np.unique(y, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  {label}: {count}")

## 2.2 降维

本部分使用**PCA**进行降维，然后使用**t-SNE**进行可视化。可视化将会使用PCA降维到50维的预处理数据

### 2.2.1 PCA和t-SNE介绍

#### PCA (主成分分析)

**PCA (Principal Component Analysis)** 是一种经典的降维技术,通过线性变换将高维数据投影到低维空间。

**基本原理**:
1. 计算数据的协方差矩阵
2. 求解协方差矩阵的特征值和特征向量
3. 选择最大的k个特征值对应的特征向量
4. 将数据投影到这k个特征向量张成的空间

**优点**:
- 计算效率高
- 能够保留数据的主要方差信息
- 去除数据冗余和噪声

**应用**:
- 第一次降维:将HOG特征降到50维,用于t-SNE可视化
- 第二次降维:保留95%的方差,用于SVM分类

#### t-SNE (t分布随机邻域嵌入)

**t-SNE (t-Distributed Stochastic Neighbor Embedding)** 是一种非线性降维算法,特别适合高维数据的可视化。

**基本原理**:
1. 在高维空间中,计算样本点之间的相似度
2. 在低维空间中,寻找一个映射使得点之间的相似度分布尽可能接近高维空间
3. 使用t分布来建模低维空间中的相似度

**优点**:
- 能够很好地保持数据的局部结构
- 适合将高维数据可视化到2D或3D空间
- 能够揭示数据的聚类结构

**注意**:
- 计算开销较大,通常先用PCA降维再使用t-SNE
- 结果受随机初始化影响,需要设置随机种子

### 2.2.2 PCA降维

第一部分降维，需要将原始维度压缩到50维，然后用于t-SNE可视化

第二部分降维，需要将原始数据的信息保留$95\%$左右，然后用于Part 3 的SVM

In [ ]:
# 第一次PCA降维: 降到50维,用于t-SNE可视化
print("=" * 60)
print("第一次PCA降维: 降到50维 (用于t-SNE可视化)")
print("=" * 60)

pca_50 = PCA(n_components=50, random_state=42)
X_pca_50 = pca_50.fit_transform(X_hog)

# 计算解释的方差比例
explained_variance_ratio_50 = pca_50.explained_variance_ratio_
cumulative_variance_ratio_50 = np.cumsum(explained_variance_ratio_50)

print(f"\n降维后的形状: {X_pca_50.shape}")
print(f"保留的总方差比例: {cumulative_variance_ratio_50[-1]:.4f} ({cumulative_variance_ratio_50[-1]*100:.2f}%)")
print(f"前10个主成分的方差比例: {explained_variance_ratio_50[:10]}")

# 可视化方差解释比例
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# 单个主成分的方差比例
ax1.bar(range(1, 51), explained_variance_ratio_50)
ax1.set_xlabel('主成分编号')
ax1.set_ylabel('方差解释比例')
ax1.set_title('各主成分的方差解释比例')
ax1.grid(True, alpha=0.3)

# 累积方差解释比例
ax2.plot(range(1, 51), cumulative_variance_ratio_50, 'b-', marker='o', markersize=3)
ax2.set_xlabel('主成分数量')
ax2.set_ylabel('累积方差解释比例')
ax2.set_title('累积方差解释比例')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0.95, color='r', linestyle='--', label='95%方差线')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 第二次PCA降维: 保留95%的方差,用于SVM分类
print("\n" + "=" * 60)
print("第二次PCA降维: 保留95%的方差 (用于SVM分类)")
print("=" * 60)

pca_95 = PCA(n_components=0.95, random_state=42)
X_pca_95 = pca_95.fit_transform(X_hog)

# 计算解释的方差比例
explained_variance_ratio_95 = pca_95.explained_variance_ratio_
cumulative_variance_ratio_95 = np.cumsum(explained_variance_ratio_95)

print(f"\n降维后的维度: {X_pca_95.shape[1]}")
print(f"降维后的形状: {X_pca_95.shape}")
print(f"保留的总方差比例: {cumulative_variance_ratio_95[-1]:.4f} ({cumulative_variance_ratio_95[-1]*100:.2f}%)")
print(f"维度压缩率: {X_pca_95.shape[1] / X_hog.shape[1]:.4f} ({X_pca_95.shape[1] / X_hog.shape[1] * 100:.2f}%)")

# 可视化
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(range(1, len(cumulative_variance_ratio_95) + 1), cumulative_variance_ratio_95, 'b-', marker='o', markersize=2)
ax.set_xlabel('主成分数量')
ax.set_ylabel('累积方差解释比例')
ax.set_title(f'PCA降维 (保留95%方差, 最终维度: {X_pca_95.shape[1]})')
ax.grid(True, alpha=0.3)
ax.axhline(y=0.95, color='r', linestyle='--', label='95%方差线')
ax.legend()
plt.tight_layout()
plt.show()

print(f"\n✓ PCA降维完成,数据已准备好用于后续的SVM分类器训练")

### 2.2.3 t-SNE可视化

In [ ]:
# 使用t-SNE将50维数据降到2维进行可视化
print("=" * 60)
print("t-SNE降维可视化")
print("=" * 60)
print("\n注意: t-SNE计算可能需要几分钟时间...")

# 为了加速演示,我们可以使用部分数据进行t-SNE可视化
# 完整实验可以使用全部数据
n_tsne_samples = 5000  # 使用5000个样本进行t-SNE可视化

# 随机选择样本
np.random.seed(42)
indices = np.random.choice(len(X_pca_50), n_tsne_samples, replace=False)
X_tsne_input = X_pca_50[indices]
y_tsne = y[indices]

print(f"使用 {n_tsne_samples} 个样本进行t-SNE可视化")
print(f"输入数据形状: {X_tsne_input.shape}")

# 应用t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_tsne_input)

print(f"t-SNE降维后的形状: {X_tsne.shape}")
print("✓ t-SNE降维完成")

# Part 3 传统模型实验

# Part 4 深度模型实验

# Part 5 总结